In [ ]:
from pyspark import SparkContext
sc = SparkContext("local[4]", "Simple App")

In [9]:
import json

In [ ]:
nCoresPerExecutor=4
indexControl=11

In [2]:
filename="genoMatTriosDamaging.txt"
filename="genoMat.txt"
varMat = sc.textFile(filename)
varMat.take(2)

[u'ENSG00000000003 X:99884731:G:C 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0',
 u'ENSG00000000005 X:99840291:A:G 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0']

In [3]:
varMat.count()

40273

In [ ]:
varMat=varMat.repartition(nCoresPerExecutor)

#Common functions

In [ ]:
def splitValues(variantData):
    variantData=variantData.split(" ")
    variantGene=variantData.pop(0)
    variantLocus=variantData.pop(0)
    
    return (variantGene,(variantLocus,variantData))

#Variant

##De Novo

In [5]:
def scoreDeNovo(variantData):
    variantData=variantData.split(" ")
    variantGene=variantData.pop(0)
    variantLocus=variantData.pop(0)
    
    score=0
    listTrios=[]
    checkValid=0
    for i in range(0,11):
        checkValid=checkValid+int(variantData[i*3+1])+int(variantData[i*3+2])
    
    if checkValid==0:                              
        for i in range(0,11):
            scoreTrio=int(variantData[(i*3)]=="1" and variantData[i*3+1]=="0" and variantData[i*3+2]=="0")
            if scoreTrio>0:
                listTrios.append(i)
                score=score+scoreTrio
            
    return ((variantGene,variantLocus),score,listTrios,variantData)

In [16]:
scores=sc.textFile(filename,8).map(scoreDeNovo).filter(lambda (k,v1,v2,v3): v1>0).takeOrdered(100000, key=lambda (k,v1,v2,v3): -v1)

In [17]:
scores=["TriosDeNovo","singleVariant",scores]

In [18]:
with open('ULB_Control_vs_NeuroDev.txt', 'w') as outfile:
    json.dump(scores, outfile)
#with open('singVarDeNovo.txt', 'w') as outfile:
#    json.dump(scores, outfile)

#Gene

##Univariate

In [ ]:
def scoreGeneUnivariate(k,variantList):
    variantList=list(variantList)
    #result=[k,(0,0,0)]
    nbVariantsCase=0
    nbVariantsControl=0
    if len(variantList)>0:
        for i in range(0,len(variantList)):
            (locus,geno)=variantList[i]
            nbVariantsCase+=sum([int(x) for x in geno[0:indexControl]])
            nbVariantsControl+=sum([int(x) for x in geno[indexControl:len(geno)]])
    
    score=nbVariantsCase-nbVariantsControl
    if score>0:
        return [k,(score,nbVariantsCase,nbVariantsControl)]
    
#    return result

In [ ]:
scores=sc.textFile(filename,nCoresPerExecutor).map(splitValues).groupByKey().map(lambda (k,v):scoreGeneUnivariate(k,v)).filter(lambda (k,(v1,v2,v3)): v1>0).takeOrdered(100, key=lambda (k,(v1,v2,v3)): -v1)

In [ ]:
scores=["neuroDev_Control","geneUnivariate",scores]

In [ ]:
with open('neuroDev_Control.txt', 'w') as outfile:
    json.dump(scores, outfile)

##Bivariate

#Common results

In [ ]:
scores

In [ ]:
scores=["neuroDev_Control","geneUnivariate",scores]

In [ ]:
with open('neuroDev_Control.txt', 'w') as outfile:
    json.dump(scores, outfile)

In [ ]:
sc.stop()